# Regd No: 22231
# Simple Neural Network


In [ ]:
import torch.nn as nn # importing torch.nn as nn

In [ ]:
!pip install torch torchvision #installing torchvision package


In [ ]:
import torch # importing torch
from torchvision import datasets # importing datasets from torchvision
from torchvision.transforms import ToTensor, Lambda #importing ToTensor and Lambda

ds_train = datasets.FashionMNIST(  # Getting dataset (training) from FashionMNIST
    root = 'data',
    train = True,
    download = True,
    transform = ToTensor(), # this converts the image into a tensor
    target_transform= Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0,torch.tensor(y), value=1))
    # This creates a tensor of zeros with a shape of (10,), indicating that there are 10 possible classes or labels. The dtype is set to float.
)

test_dataset = datasets.FashionMNIST(  # Getting dataset (testing) from FashionMNIST
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor(), # this converts the image into a tensor
    target_transform= Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0,torch.tensor(y), value=1))
    # This creates a tensor of zeros with a shape of (10,), indicating that there are 10 possible classes or labels. The dtype is set to float.
)

100%|██████████| 26421880/26421880 [00:01<00:00, 17051229.51it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 310182.79it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5474453.51it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 16271497.36it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
# this chunk is used to get the validation datset from the training dataset
from torch.utils.data import random_split # importing random_split


train_size = len(ds_train) # taking length of the training set
val_size = int(0.1 * train_size)  # Taking 10% of the training set for validation

train_dataset, val_dataset = random_split(ds_train, [train_size - val_size, val_size]) # seperating into 90% of training into new training and 10% of training to validation

print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")


Training set size: 54000
Validation set size: 6000


In [ ]:
print(f"Test set size: {len(test_dataset)}")

Test set size: 10000


In [ ]:
# Define a simple neural network architecture
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.linear1 = nn.Linear(in_features=28*28, out_features=128)  # Input size 784 (28x28), output size 128
        self.linear2 = nn.Linear(in_features=128, out_features=64)   # Input size 128, output size 64
        self.linear3 = nn.Linear(in_features=64, out_features=10)    # Input size 64, output size 10 for 10 classes

    def forward(self, x): # forward prop function
        x = x.view(x.size(0),-1)
        x = torch.relu(self.linear1(x))
        x = torch.relu(self.linear2(x))
        x = torch.softmax(self.linear3(x),dim = 1)
        return x

# Create an instance of the SimpleNet
simple_net = SimpleNet()

# Now we will train it , validate it , test it

## Choosing the loss function and optimizer

In [ ]:
import torch.optim as optim #importing optim

In [ ]:
criterion = nn.CrossEntropyLoss()  # Taking Cross Entropy Loss as the Loss function
optimizer = optim.SGD(simple_net.parameters(), lr=0.01)  # optimizer is SGD stochastic gradient descent


# Creating DataLoaders
Here Data Loaders are iteratable dataobject , where iterating over the batches and the data is fed into the model for processing.

In [ ]:
from torch.utils.data import DataLoader # importing dataloader

In [ ]:
# loading all the datasets into dataloader for feeding into the model by batches

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


# Training and Validating the Model

In [ ]:
def trainval_loop(train_loader, val_loader, simple_net, criterion, optimizer): #parameters for the function
    # Set the model to training mode before entering loops
    simple_net.train()

    # Training loop
    size = len(train_loader.dataset)
    for batch, (X, y) in enumerate(train_loader):
        # Compute prediction and loss (training)
        pred = simple_net(X)
        loss = criterion(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad() # this is done to deny the involement of the previous gradients interference with the current ones

        if batch % 100 == 0:
            current = batch * len(X)
            print(f"Training loss: {loss.item():>7f}  [{current:>5d}/{size:>5d}]") #printing training loss

    # Set the model to evaluation mode before validation
    simple_net.eval()

    # Validation loop
    val_loss = 0
    val_size = len(val_loader.dataset)
    with torch.no_grad():  # No need to compute gradients during validation
        for batch, (X, y) in enumerate(val_loader):
            # Compute prediction and loss
            pred = simple_net(X)
            loss = criterion(pred, y)
            val_loss += loss.item()

            if batch % 100 == 0:
                current = batch * len(X)
                print(f"Validation loss: {loss.item():>7f}  [{current:>5d}/{val_size:>5d}]") #printing validation loss

    avg_val_loss = val_loss / len(val_loader)
    print(f"Avg. Validation loss: {avg_val_loss:>7f}") #printing average validation loss





# Testing the Model



In [ ]:
# this chunk is important for testing the model
def test_loop(test_loader, simple_net, criterion): # function test_loop taking these respective parameters
    simple_net.eval() # setting the model to eval
    size = len(test_loader.dataset)
    num_batches = len(test_loader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in test_loader:
            pred = simple_net(X)
            test_loss += criterion(pred, y).item()

            # Calculate the number of correct predictions in this batch
            pred_labels = pred.argmax(dim=1)
            y= y.argmax(dim=1)
            #print(pred_labels,y)
            correct += (pred_labels == y).sum().item()

    test_loss /= num_batches
    accuracy = correct / size
    print(f"Test Error: \n Accuracy: {(100*accuracy):.2f}%, Avg loss: {test_loss:.8f} \n") # printint the Accuracy (TEST) and the test_loss




# Now Training and Validation

In [ ]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    trainval_loop(train_loader,val_loader, simple_net, criterion, optimizer) # running train_val loop by 10 times
print("Done!")

Epoch 1
-------------------------------
Training loss: 2.303422  [    0/54000]
Training loss: 2.304411  [ 6400/54000]
Training loss: 2.302629  [12800/54000]
Training loss: 2.298604  [19200/54000]
Training loss: 2.298018  [25600/54000]
Training loss: 2.296574  [32000/54000]
Training loss: 2.297565  [38400/54000]
Training loss: 2.295180  [44800/54000]
Training loss: 2.294214  [51200/54000]
Validation loss: 2.294480  [    0/ 6000]
Avg. Validation loss: 2.293875
Epoch 2
-------------------------------
Training loss: 2.291824  [    0/54000]
Training loss: 2.294116  [ 6400/54000]
Training loss: 2.289575  [12800/54000]
Training loss: 2.287416  [19200/54000]
Training loss: 2.283454  [25600/54000]
Training loss: 2.285635  [32000/54000]
Training loss: 2.280832  [38400/54000]
Training loss: 2.265028  [44800/54000]
Training loss: 2.270756  [51200/54000]
Validation loss: 2.271298  [    0/ 6000]
Avg. Validation loss: 2.268808
Epoch 3
-------------------------------
Training loss: 2.273753  [    0/54

# Testing

In [ ]:
test_loop(test_loader, simple_net, criterion) #test_loop running and checking the results


Test Error: 
 Accuracy: 73.61%, Avg loss: 1.75831701 

